In [1]:
%run SectionAnalyser.ipynb
%run Util.ipynb

sectionAnalyser = SectionAnalyser(scale=1, short=10, mid=30)

sectionAnalyser.build_sections_file_path('single_level',level_day)

'/Users/xinwang/stock/data/sections/single_level_10_30_Day_1_all_sections_V0.8.txt'

In [6]:
%run SectionAnalyser.ipynb
%run Util.ipynb

sectionAnalyser = SectionAnalyser(scale=3, short=10, mid=30)

week_sections_df = sectionAnalyser.get_all_sections_of_all_symbols(level_week)
week_sections_df


day_sections_df = sectionAnalyser.get_all_sections_of_all_symbols(level_day)
sectionAnalyser.get_price_incre_percent_report(day_sections_df)


start to analyse Week level SZ#000001 all cross sections...
start to analyse Week level SZ#000652 all cross sections...
start to analyse Week level SZ#000960 all cross sections...
get_all_sections_of_all_symbols done total:46
start to analyse Day level SZ#000001 all cross sections...
start to analyse Day level SZ#000652 all cross sections...
start to analyse Day level SZ#000960 all cross sections...
get_all_sections_of_all_symbols done total:248
threshold 10, over count size:52 total size:248 rate 20.967741935483872
threshold 20, over count size:27 total size:248 rate 10.887096774193548
threshold 30, over count size:19 total size:248 rate 7.661290322580645
threshold 40, over count size:13 total size:248 rate 5.241935483870968
threshold 50, over count size:9 total size:248 rate 3.629032258064516
threshold 60, over count size:8 total size:248 rate 3.225806451612903
threshold 70, over count size:7 total size:248 rate 2.8225806451612905
threshold 80, over count size:6 total size:248 rate 2

In [ ]:
%run SectionAnalyser.ipynb
%run Util.ipynb
%run Metric.ipynb

short=30
mid=60
sectionAnalyser = SectionAnalyser(100, short, mid)


day_sections_df = sectionAnalyser.get_all_sections_of_all_symbols(level_day)
day_sections_df['entry_to_max_increase_per'] = 0

for key,row in day_sections_df.iterrows():
    metric = Metric(row['symbol'],level_day,short,mid)
    
    start_date = row['s_date']
    end_date = row['e_date']
    
    max_price_series = metric.get_max_price_date(start_date,end_date)
    highest_price = max_price_series['high']
    entry_price = metric.get_cur_price(start_date)
    
    entry_to_max_increase_percent = round(100*(highest_price-entry_price)/entry_price,2)
    
    day_sections_df.loc[key,'entry_to_max_increase_per'] = entry_to_max_increase_percent
    
    metric.clean_metric()


day_sections_df[['symbol','s_date','e_date','entry_to_max_increase_per']]

In [ ]:
%run ChanBiAnalyser.ipynb

part=day_sections_df[['symbol','s_date','e_date','entry_to_max_increase_per']]

all_chan_bi_df = pd.DataFrame()
for key,row in day_sections_df.iterrows():
    chanBiAnalyser = ChanBiAnalyser(row['symbol'],level_day,row['s_date'],row['e_date'],30,60)
    chan_bi_df = chanBiAnalyser.get_chan_bi()
    
    all_chan_bi_df = pd.concat([chan_bi_df,all_chan_bi_df], ignore_index=True, sort=False)
    
all_chan_bi_df


In [ ]:

bins=[-100,50,100,20000]
cuts = pd.cut(part['entry_to_max_increase_per'],bins)

total = part.shape[0]
print('total ' + str(total))


df = pd.DataFrame(columns=['bin','max_increase_per'])
df['bin'] = pd.value_counts(cuts).index
df['max_increase_per'] = pd.value_counts(cuts).values
df['max_increase_per'] = round(100*df['max_increase_per']/total,2)

df

In [ ]:
%run SectionAnalyser.ipynb
%run Util.ipynb

sectionAnalyser = SectionAnalyser(scale=3, short=30, mid=60)
bi_df = sectionAnalyser.split_sections_into_chan_bi(level_day)
bi_df

In [ ]:
resonate_df = sectionAnalyser.get_cross_resonate_on_week_and_day()
resonate_df

In [ ]:
def report(df):
    avg_percent = df['d_price_incre'].mean()
    print('avg d_price_incre:' + str(avg_percent))
    
    for threshold in [10,20,40,60,80]:
        over_count = df[df['d_price_incre'] > threshold].shape[0]
        total = df.shape[0]

        if(total>0):
            rate = 100*over_count/total
            print('threshold '+ str(threshold) + ', over count size:'+ str(over_count) +
                  ' total size:'+str(total) + ' rate ' +str(rate))    

                    
report(resonate_df[(resonate_df['d_ma_mid_dire']=='Up')])





In [ ]:
d_boll_up_resonate_df = resonate_df[(resonate_df['d_boll']==True) & (resonate_df['d_ma_short_dire']=='Up')]

for threshold in [10,20,40,60,80]:
            over_count = d_boll_up_resonate_df[d_boll_up_resonate_df['d_price_incre'] > threshold].shape[0]
            total = d_boll_up_resonate_df.shape[0]

            if(total>0):
                rate = 100*over_count/total
                print('threshold '+ str(threshold) + ', over count size:'+ str(over_count) +
                      ' total size:'+str(total) + ' rate ' +str(rate))